### Script for transforming UCF-data 
#### Description
This script transforms the UCF-training data, so that it's compatible with "iSeeBetterTrain.py"-script. It creates a new directory "UCF/train_formatted" with new PNGs and JPGs. We only use the PNGs for training.

In [17]:
import os
import sys
from shutil import copyfile
from PIL import Image

In [51]:
'''
Reformat training-data
'''
def format_training_data():
    
    base_path = os.getcwd()
    
    # Create Dict
    images = os.listdir('./UCF/train')
    image_dict = {}
    
    for image in images:
        if '.ipynb' not in image:
            splitted_image = image.split('_')[0]
            image_dict[splitted_image] = []
    
    # Format data
    for image in images:
        if '.ipynb' not in image:
            splitted_image = image.split('_')
            image_number = splitted_image[-1].replace('.jpg', '')
            image_dict[splitted_image[0]].append({
                'core': splitted_image[0] + '_' + splitted_image[1],
                'key': int(image_number),
                'original_name': image
            })
            
    for key in image_dict:
        image_dict[key].sort(key=lambda x: x['key'])
        dir_name = 'UCF/train_formatted/' + key
        if os.path.isdir(dir_name) == False:
            os.makedirs(dir_name)
        
        # Reset keys
        for idx in range(len(image_dict[key])):
            image_dict[key][idx]['key'] = idx
        

        for image in image_dict[key]:
            old_file_name = 'UCF/train/' + image['original_name']
            new_file_name = 'UCF/train_formatted/' + key + '/' +  'im' + str(image['key']) + '.jpg'
            new_file_name_2 = 'UCF/train_formatted/' + key + '/' +  'im-' + str(image['key']) + '.jpg'
            copyfile(old_file_name, new_file_name)
            copyfile(old_file_name, new_file_name_2)
            im = Image.open(new_file_name)
            png_name = new_file_name.replace('.jpg', '.png')
            im.save(png_name)
            
            im = Image.open(new_file_name_2)
            png_name = new_file_name_2.replace('.jpg', '.png')
            im.save(png_name)
            
            
    return image_dict

'''
Reformat testing-data
'''
def format_testing_data():
    
    base_path = os.getcwd()
    
    # Create Dict
    images = os.listdir('./UCF/downsampled_test_set/video_images')
    image_dict = {}
    
    for image in images:
        if '.ipynb' not in image:
            splitted_image = image.split('_')[0]
            image_dict[splitted_image] = []
    
    # Format data
    for image in images:
        if '.ipynb' not in image:
            splitted_image = image.split('_')
            image_number = splitted_image[-1].replace('.jpg', '')
            image_dict[splitted_image[0]].append({
                'core': splitted_image[0] + '_' + splitted_image[1],
                'key': int(image_number),
                'original_name': image
            })
    
    images_names_for_txt_file = []
    for key in image_dict:
        image_dict[key].sort(key=lambda x: x['key'])
        dir_name = 'UCF/test_formatted/' + key
        if os.path.isdir(dir_name) == False:
            os.makedirs(dir_name)
        
        # Reset keys
        for idx in range(len(image_dict[key])):
            image_dict[key][idx]['key'] = idx
        

        for image in image_dict[key]:
            old_file_name = 'UCF/downsampled_test_set/video_images/' + image['original_name']
            new_file_name = 'UCF/test_formatted/' + key + '/' + str(image['key']).zfill(3) + '.jpg'
            copyfile(old_file_name, new_file_name)
            im = Image.open(new_file_name)

            im.save(new_file_name.replace('.jpg', '.png'))
            file_name_in_test_list = 'test_formatted/' + key + '/' + str(image['key']).zfill(3) + '.png'
            images_names_for_txt_file.append(file_name_in_test_list)
    
    with open('UCF/ucf_downsampled_testlist.txt', 'w') as f:
        for item in images_names_for_txt_file:
            f.write("%s\n" % item)
    return image_dict

#dict_with_data = main()
#testing_dict = format_testing_data()
training_dict = format_training_data()
#print(str(1).zfill(3))

### Script for training data
#### Description
The following starts the training based on the data formatted above

In [3]:
!sudo apt-get -y install gcc
!sudo apt update
!sudo apt install -y libgl1-mesa-glx
!pip3 install scikit-image cython opencv-python
!pip3 install -r requirements.txt
!cd pyflow && python setup.py build_ext -i && python demo.py && cp pyflow*.so ..

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gcc is already the newest version (4:7.4.0-1ubuntu2.3).
0 upgraded, 0 newly installed, 0 to remove and 61 not upgraded.
Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease         
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Reading package lists... Done0m0m                  
Building dependency tree       
Reading state information..

1

Next, let's run the training script

In [ ]:
!python iSeeBetterTrain.py --nFrames=9 --nEpochs=50000 --snapshots=10 --patch_size=32 --batchSize=1 --threads=1 --data_dir="./UCF" --file_list="ucf_trainlist.txt" --upscale_factor=4 --save_folder="weights/UCF"

[    INFO] ==> Loading datasets
Training samples chosen: ucf_trainlist.txt
[    INFO] # of Generator parameters: 12775399
[    INFO] # of Discriminator parameters: 5215425
[    INFO] # of CUDA devices detected: 1
[    INFO] Using CUDA device #: 0
[    INFO] CUDA device name: Tesla V100-SXM2-16GB
[    INFO] Generator Loss: L1 Loss
[    INFO] ------------- iSeeBetter Network Architecture -------------
[    INFO] ----------------- Generator Architecture ------------------
[    INFO] Net(
  (feat0): ConvBlock(
    (conv): Conv2d(3, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (act): PReLU(num_parameters=1)
  )
  (feat1): ConvBlock(
    (conv): Conv2d(8, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (act): PReLU(num_parameters=1)
  )
  (DBPN): Net(
    (feat1): ConvBlock(
      (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
      (act): PReLU(num_parameters=1)
    )
    (up1): UpBlock(
      (up_conv1): DeconvBlock(
        (deconv): ConvTranspose2d(

## Script for testing data
## mango

In [ ]:
# !python iSeeBetterTest.py --model="weights/netG_epoch_4_1_FullDataSet.pth" --file_list="foliage.txt" --threads=0 --gpu_mode
!python iSeeBetterTest.py --nFrames=9 --model="weights/netG_epoch_4_11359.pth" --data_dir="./UCF" --file_list="ucf_downsampled_testlist.txt" --threads=0 --gpu_mode --upscale_only
# RBPN_4x_F11_NTIRE2019s

Namespace(chop_forward=False, data_dir='./UCF', debug=False, file_list='ucf_downsampled_testlist.txt', future_frame=True, gpu_mode=True, gpus=1, model='weights/netG_epoch_4_11359.pth', model_type='RBPN', nFrames=9, other_dataset=True, output='Results/', residual=False, seed=123, testBatchSize=1, threads=0, upscale_factor=4, upscale_only=True)
Using GPU mode
==> Loading datasets
==> Building model  RBPN
[    INFO] ------------- iSeeBetter Network Architecture -------------
[    INFO] ----------------- Generator Architecture ------------------
[    INFO] DataParallel(
  (module): Net(
    (feat0): ConvBlock(
      (conv): Conv2d(3, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): PReLU(num_parameters=1)
    )
    (feat1): ConvBlock(
      (conv): Conv2d(8, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (act): PReLU(num_parameters=1)
    )
    (DBPN): Net(
      (feat1): ConvBlock(
        (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1))
      

In [19]:
# TODO: Map number to original image name
#testing_dict

{'Fighting031': [{'core': 'Fighting031_x264',
   'key': 0,
   'original_name': 'Fighting031_x264_1.jpg'},
  {'core': 'Fighting031_x264',
   'key': 1,
   'original_name': 'Fighting031_x264_3.jpg'},
  {'core': 'Fighting031_x264',
   'key': 2,
   'original_name': 'Fighting031_x264_11.jpg'},
  {'core': 'Fighting031_x264',
   'key': 3,
   'original_name': 'Fighting031_x264_13.jpg'},
  {'core': 'Fighting031_x264',
   'key': 4,
   'original_name': 'Fighting031_x264_20.jpg'},
  {'core': 'Fighting031_x264',
   'key': 5,
   'original_name': 'Fighting031_x264_22.jpg'},
  {'core': 'Fighting031_x264',
   'key': 6,
   'original_name': 'Fighting031_x264_24.jpg'},
  {'core': 'Fighting031_x264',
   'key': 7,
   'original_name': 'Fighting031_x264_28.jpg'},
  {'core': 'Fighting031_x264',
   'key': 8,
   'original_name': 'Fighting031_x264_32.jpg'},
  {'core': 'Fighting031_x264',
   'key': 9,
   'original_name': 'Fighting031_x264_33.jpg'},
  {'core': 'Fighting031_x264',
   'key': 10,
   'original_name': 'F

In [62]:

def rename_to_original_naming_convention(directory, testing_dict):
    images = os.listdir('./Results/UCF/' + directory)
    dir_name = 'Results/UCF/' + directory + '_renamed'
    if os.path.isdir(dir_name) == False:
        os.makedirs(dir_name)

    for image in images:
        if '.png' in image:
            splitted_name = image.split('_')
            dict_key = splitted_name[0]
            key = splitted_name[-1].replace('.png', '')
            key = int(key)
            
            mapping_element = list(filter(lambda image: image['key'] == key, testing_dict[dict_key]))
            original_name = mapping_element[0]['original_name']
            
            old_file_name = 'Results/UCF/' + directory + '/' + image
            new_file_name = dir_name + '/' + original_name.replace('.jpg', '.png')
            copyfile(old_file_name, new_file_name)

rename_to_original_naming_convention('ucf_downsampled_testlist_4x', testing_dict)